<a href="https://colab.research.google.com/github/ymuto0302/base2_2020/blob/master/opendata4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オープンデータの活用(4) : Google's COVID-19 Community Mobility Reports

これまで日本国内のデータを利用してきたが，今回は新型コロナウイルス関連で世界中をカバーする "Google's COVID-19 Community Mobility Reports" を利用する。

Community Mobility Reports とは，(天下の)Google がロケーション履歴を元に集計したレポートであり，「食料品店，乗換駅などの特定の場所を訪れた人数の変化」をオープンデータ化している。


## Google Community Mobility Reports からのデータ入手
下記の COVID-19: コミュニティ モビリティ レポートからデータを入手しよう。

https://www.google.com/covid19/mobility/

ページ内の中段にある「全世界のCSVをダウンロード」をクリックし，ファイル "Global_Mobility_Report.csv" を Google Drive に置いておく。


## Google Drive のマウント & plotly express のインストール
plotly express は前回も用いた「可視化ライブラリ」である。

In [1]:
# Google Drive のマウント
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
# 可視化の準備：PlotlyExpress のインストール
!pip install plotly_express

## Google Community Mobility Reports のデータを眺めてみる
下記のとおり 588539行×13列の膨大なデータである。

ここで "country_region_code" が国コードを表し，日本は "JP" である。
また，"sub_region_1" には都道府県名や州名が入る。

右の方を眺めるとデータ観測日である date に続き，特定の場所（食料品店、公園など）を訪れた人数の変化が示されている。「人数の変化」は次の６項目からなる。
- retail_and_recreation_percent_change_from_baseline # 小売、娯楽(対象: レストラン、カフェ、ショッピング センター、テーマパーク、博物館、図書館、映画館など)
- grocery_and_pharmacy_percent_change_from_baseline # 食料品店、薬局(対象: 食料品店、食品問屋、青果市場、高級食料品店、ドラッグストア、薬局など)
- parks_percent_change_from_baseline # 公園(対象: 地域の公園、国立公園、公共のビーチ、マリーナ、ドッグパーク、広場、庭園など)
- transit_stations_percent_change_from_baselin # 乗換駅(対象: 公共交通機関の拠点（例: 地下鉄、バス、電車の駅）など)
- workplaces_percent_change_from_baseline # 職場(対象: 職場)
- residential_percent_change_from_baseline # 住宅(対象: 住居)

また，「人数の変化」はパーセント表示であり，Google Community Mobility Reports によると，その基準値は「2020年1月3日から2020年2月6日までの期間の中央値」を用いている。

In [18]:
import pandas as pd

# Google's COVID-19 Community Mobility Reports (COVID-19 コミュニティ モビリティ レポート) の読み込み
df = pd.read_csv("/content/drive/My Drive/BASE2/OpenData/Global_Mobility_Report.csv", encoding="utf-8")
df

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.



,country_region_code,country_region,sub_region_1,sub_region_2,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-15,0.0,4.0,5.0,0.0,2.0,1.0
1,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-16,1.0,4.0,4.0,1.0,2.0,1.0
2,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-17,-1.0,1.0,5.0,1.0,2.0,1.0
3,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-18,-2.0,1.0,5.0,0.0,2.0,1.0
4,AE,United Arab Emirates,NaN,NaN,NaN,NaN,2020-02-19,-2.0,0.0,4.0,-1.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
588534,ZW,Zimbabwe,Midlands Province,NaN,ZW-MI,NaN,2020-06-19,NaN,NaN,NaN,NaN,-2.0,NaN
588535,ZW,Zimbabwe,Midlands Province,NaN,ZW-MI,NaN,2020-06-20,NaN,NaN,NaN,NaN,23.0,NaN
588536,ZW,Zimbabwe,Midlands Province,NaN,ZW-MI,NaN,2020-06-21,NaN,NaN,NaN,NaN,24.0,NaN
588537,ZW,Zimbabwe,Midlands Province,NaN,ZW-MI,NaN,2020-06-22,NaN,NaN,NaN,NaN,-5.0,NaN


## 日本のデータのみ取り出す
全世界のデータから日本のデータのみを取り出すには pandas の機能を利用する。

データを絞り込んだ結果，6240件のデータが得られた。

In [19]:
# 日本のデータのみ絞り込む (# データ数 = 6240件)
df = df[df['country_region_code'] == 'JP']
df

,country_region_code,country_region,sub_region_1,sub_region_2,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
115406,JP,Japan,NaN,NaN,NaN,NaN,2020-02-15,-1.0,4.0,7.0,1.0,4.0,0.0
115407,JP,Japan,NaN,NaN,NaN,NaN,2020-02-16,-9.0,-6.0,-35.0,-10.0,-2.0,2.0
115408,JP,Japan,NaN,NaN,NaN,NaN,2020-02-17,-2.0,1.0,-5.0,-1.0,0.0,1.0
115409,JP,Japan,NaN,NaN,NaN,NaN,2020-02-18,1.0,2.0,1.0,-2.0,0.0,1.0
115410,JP,Japan,NaN,NaN,NaN,NaN,2020-02-19,0.0,2.0,8.0,-3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
121641,JP,Japan,Yamanashi,NaN,JP-19,NaN,2020-06-19,-12.0,-6.0,-34.0,-28.0,-12.0,8.0
121642,JP,Japan,Yamanashi,NaN,JP-19,NaN,2020-06-20,-6.0,7.0,26.0,-12.0,-9.0,4.0
121643,JP,Japan,Yamanashi,NaN,JP-19,NaN,2020-06-21,-4.0,6.0,19.0,-10.0,-6.0,4.0
121644,JP,Japan,Yamanashi,NaN,JP-19,NaN,2020-06-22,-13.0,-8.0,-38.0,-26.0,-9.0,7.0


## サンプルプログラム
山口県のデータのみ抽出し，可視化するプログラムを以下に示す。

可視化した結果を眺めたところ，以下の考察が可能である。
- 「乗換駅」への人の移動は 3月3日に底(-60%超)を打ち，その後は回復している。
- 「食料品店・薬局」では生活必需品を購入することから，3月3日に -21% と落ち込んだものの，全期間を通じて増減は見られない。

ところで，3月3日に複数の指標が底を打っているのは「大阪市内のライブ参加者の感染相次ぐ」のが原因でしょうか？
https://www.nippon.com/ja/japan-data/h00730/

In [17]:
# COVID-19: コミュニティ モビリティ レポートの可視化
# https://www.google.com/covid19/mobility/
# データファイル名は Global_Mobility_Report.csv (2020.06.30 アクセス)

import pandas as pd

# Google's COVID-19 Community Mobility Reports (COVID-19 コミュニティ モビリティ レポート) の読み込み
df = pd.read_csv("/content/drive/My Drive/BASE2/OpenData/Global_Mobility_Report.csv", encoding="utf-8")

'''
各列の名称は以下のとおり：
retail_and_recreation_percent_change_from_baseline # 小売、娯楽(対象: レストラン、カフェ、ショッピング センター、テーマパーク、博物館、図書館、映画館など)
grocery_and_pharmacy_percent_change_from_baseline # 食料品店、薬局(対象: 食料品店、食品問屋、青果市場、高級食料品店、ドラッグストア、薬局など)
parks_percent_change_from_baseline # 公園(対象: 地域の公園、国立公園、公共のビーチ、マリーナ、ドッグパーク、広場、庭園など)
transit_stations_percent_change_from_baselin # 乗換駅(対象: 公共交通機関の拠点（例: 地下鉄、バス、電車の駅）など)
workplaces_percent_change_from_baseline # 職場(対象: 職場)
residential_percent_change_from_baseline # 住宅(対象: 住居)
'''

# 列名が長いため，短縮かつ日本語化する
# (メモ) rename の書式 : columns={変更前A:変更後A, 変更前B:変更後B}
df = df.rename(columns={'retail_and_recreation_percent_change_from_baseline':'小売・娯楽',
                        'grocery_and_pharmacy_percent_change_from_baseline': '食料品店・薬局',
                        'parks_percent_change_from_baseline': '公園',
                        'transit_stations_percent_change_from_baseline': '乗換駅',
                        'workplaces_percent_change_from_baseline': '職場',
                        'residential_percent_change_from_baseline': '住宅'})

# 日本のデータのみ絞り込む (# データ数 = 6240件)
df = df[df['country_region_code'] == 'JP']
          
# 山口県のデータのみ絞り込む　(データ数 = 130件)
df = df[df['sub_region_1'] == 'Yamaguchi']

# 山口県における人の移動状況を折れ線グラフで表示
import plotly_express as px
df_melt = df.melt(id_vars='date', value_vars=["小売・娯楽", "食料品店・薬局", "公園", "乗換駅", "職場", "住宅"])
fig = px.line(df_melt, x="date", y="value", color='variable',
              title="Google Mobility Report")

# (メモ)Anaonda 3 では以下の命令でグラフを描ける
# fig = px.line(df, x="date", y=["小売・娯楽", "食料品店・薬局", "公園", "乗換駅", "職場", "住宅"],
#              title="Google Mobility Report")

fig.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning:

Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.

